## Preliminaries
실습 환경  
- gpu : 1 gtx2080ti  
- ram : 64gb  
- cpu : i7-7700  
- Windows (Linux recommended)  

- cuda 10
- python 3.6
- pytorch 1.3
- numpy < 1.17  
- pandas (latest version)
- (option)tensorflow 1.14 (tensorboard를 이용할 경우)

In [51]:
!pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git

  Cloning https://github.com/ildoonet/pytorch-gradual-warmup-lr.git to c:\users\jhlee\appdata\local\temp\pip-req-build-v5u9lsvd
  Created wheel for warmup-scheduler: filename=warmup_scheduler-0.1-cp36-none-any.whl size=3586 sha256=9477904bd247ae8553c2d2dfee9e7c4648db7169fb2174c0c8df1a7d47d54a16
  Stored in directory: C:\Users\JHLee\AppData\Local\Temp\pip-ephem-wheel-cache-naw3673_\wheels\b7\24\83\d30234cc013cff538805b14df916e79091f7cf9ee2c5bf3a64
Successfully built warmup-scheduler


  Running command git clone -q https://github.com/ildoonet/pytorch-gradual-warmup-lr.git 'C:\Users\JHLee\AppData\Local\Temp\pip-req-build-v5u9lsvd'


In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from warmup_scheduler import GradualWarmupScheduler
import pickle
import os
from collections import Counter
from itertools import islice
import pandas as pd
import numpy as np

---
# 1. 실습 Dataset 소개 및 전처리
### 학습에 이용할 데이터셋 : HDFS(하둡 분산 처리 시스템)  
Open dataset인 [Hadoop Distributed File System(HDFS)](https://github.com/logpai/loghub/blob/master/HDFS/HDFS_2k.log) 데이터셋은 하둡 분산처리 시스템의 로그로 이루어져 있습니다. 데이터셋 중 약 3%는 이상 상태에 대한 로그입니다.  
데이터는 아래와 같은 **'로그 키'**들로 이루어져 있습니다. 로그 키 내부에는 시스템이 수행하고 있는 일에 대한 설명이 텍스트 형식으로 적혀 있습니다.  
  
`081109 204525 512 INFO dfs.DataNode$PacketResponder: PacketResponder 2 for block blk_572492839287299681 terminating`
  

로그 내의 `block blk_572492839287299681`와 같은 부분을 **block_id**로 표시하도록 하겠습니다.  
block_id는 해당 로그가 발생한 사건의 고유 정보를 의미합니다. 같은 block_id를 갖는 로그 키는 모두 같은 사건에 대한 정보를 담고 있습니다.  
  
예를 들어 어떤 서버에 사용자가 로그인을 하는 사건이 아래와 같은 세 가지 이벤트로 이루어져 있다고 생각해보겠습니다.   

>1) 로그인 할 때 사용자가 시스템에 아이디, 패스워드를 보냄  
2) 시스템이 아이디, 패스워드를 DB 내의 정보와 매칭함  
3) 시스템이 최종적으로 로그인을 허가    

사용자가 로그인을 할 경우 같은 block_id를 갖는 세 개의 로그 키가 등장하게 됩니다.  
  
위 링크에서 확인할 수 있는 원본 데이터셋은 block_id에 대한 정렬 없이 단순히 시간 순서대로 로그들이 나열되어 있습니다.  
아래는 HDFS 데이터셋에 존재하는 기존 로그 파일의 일부로 block_id가 **blk_8229193803249955061**인 로그 키가 3개, **blk_-1000245396392748444**인 로그 키가 2개 있는 로그 시퀀스입니다.  

```
081109 204015 308 INFO dfs.DataNode$PacketResponder: PacketResponder 2 for block blk_8229193803249955061 terminating

081109 204106 329 INFO dfs.DataNode$PacketResponder: PacketResponder 2 for blk_-1000245396392748444 terminating

081109 204132 26 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.43.115:50010 is added to blk_8229193803249955061 size 67108864

081109 204324 34 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.203.80:50010 is added to blk_-1000245396392748444 size 67108864

081109 204453 34 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.250.11.85:50010 is added to blk_8229193803249955061 size 67108864

081109 204525 512 INFO dfs.DataNode$PacketResponder: PacketResponder 2 for block blk_8229193803249955061 terminating


```

### 1-1) 데이터 전처리 1 : Log Parser
Log Parser는 비정형 로그 데이터를 정형화된 템플릿으로 변환하는 역할을 수행합니다.  
본 데이터셋에 대해서는 트리 기반 파서인 [Drain Log Parser](https://github.com/logpai/logparser/tree/master/logparser/Drain)를 이용합니다.(본 튜토리얼에서는 Log Parser에 대한 자세한 학습 과정 및 설명은 생략합니다.)  
데이터에 대해 Parser를 학습하면 비정형의 로그 텍스트들에 대하여 정형화된 템플릿 규칙을 만들어냅니다.  
간단하게 로그 텍스트에서 변하지 않는 부분을 찾아내는 것으로 생각할 수 있습니다.     
아래는 Drain Log Parser를 HDFS 데이터셋에 대해서 학습을 시켜 추출한 템플릿입니다.  

In [6]:
templates = pd.read_csv('./data/train_templates.csv')
templates.head()

,EventId,EventTemplate,Occurrences
0,09a53393,Receiving block <*> src: <*> dest: <*>,1341924
1,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,446578
2,dc2c74b7,PacketResponder <*> for block <*> terminating,1339734
3,e3df2680,Received block <*> of size <*> from <*>,1339734
4,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap upd...,1343739


Log Parser를 raw data에 대해 적용한다면 위와 같은 템플릿과 함께 템플릿에 생략된 block_id 등의 정보도 함께 추출되게 됩니다.  
해당 템플릿에 따르면 위 예시의 로그 시퀀스는 아래와 같이 변하며 변하기 전 로그 키에 있던 block_id가 함께 표시되었습니다.  

```
2, blk_8229193803249955061
2, blk_-1000245396392748444
4, blk_8229193803249955061
4, blk_-1000245396392748444
4, blk_8229193803249955061
2, blk_8229193803249955061


```

추출된 파라미터 중 block_id를 이용해 동일한 block_id별로 그룹을 묶으면 아래와 같습니다.  

```
{'blk_8229193803249955061': 2 4 4 2,
 'blk_-1000245396392748444' : 2 4}


```

data 폴더 내의 **train.pkl**과 **test.pkl**은 미리 학습된 Drain Parser를 통해 위와 같은 전처리가 완료된 파일입니다.  
각 로그 시퀀스들은 위처럼 block_id와 로그 키 템플릿의 시퀀스로 이루어진 `{block_id : log sequence}` 형식의 python dictonary입니다.  
예시는 아래와 같습니다.

In [7]:
with open('./data/train.pkl', 'rb') as f:
    train = pickle.load(f)
with open('./data/test.pkl', 'rb') as f:
    test = pickle.load(f)    

In [8]:
list(test.items())[:5]

[('blk_-1000195927844309648',
  [0, 0, 0, 3, 17, 18, 17, 18, 17, 18, 6, 6, 6, 14, 14, 14, 16, 16, 16]),
 ('blk_-1000245396392748444',
  [3, 0, 0, 0, 6, 6, 6, 17, 18, 17, 18, 17, 18, 2, 14, 14, 14, 16, 16, 16]),
 ('blk_-1000285592763698141',
  [0,
   0,
   3,
   0,
   17,
   18,
   17,
   18,
   6,
   6,
   6,
   17,
   18,
   2,
   7,
   7,
   11,
   7,
   7,
   11,
   2,
   14,
   14,
   14,
   16,
   16,
   16]),
 ('blk_-1000297946873432694',
  [0, 0, 0, 3, 17, 18, 17, 18, 17, 18, 6, 6, 6, 14, 14, 14, 16, 16, 16]),
 ('blk_-1000321454365365927',
  [0, 0, 0, 3, 17, 18, 17, 18, 17, 18, 6, 6, 6, 2, 14, 14, 14, 16, 16, 16])]

### 1-2) 데이터 전처리 2 : Data Encoding
일반적인 회귀분석이나 이미지 처리와 달리 텍스트나 이번 데이터와 같은 불연속형 데이터는 모델에 넣기 위해 인코딩 과정이 별도로 필요합니다.  
본 과정에서는 이를 위해 간단한 `Vocab` 모듈을 제작해 사용합니다.  
Vocab 모듈은 input 데이터 내의 각 원소들을 embedding matrix에서 해당 원소에 해당하는 row에 매칭시키는 역할을 수행합니다.  

In [9]:
class Vocab:
    ''' 데이터 인코딩에 필요한 vocab을 만드는 모듈 '''
    def __init__(self, vocab_path):
        # 모델에 필요한 special token을 정의합니다.
        self.special_tokens = ['<PAD>', '<BOS>', '<EOS>', '<UNK>']
        self.vocab_path = vocab_path
        self.data = None
        self.vocab = None
        if os.path.isfile(vocab_path):
            self.load_vocab() # vocab을 제작할 데이터 로드

    def load_data(self, data_path):
        with open(data_path, 'rb') as f:
            data = pickle.load(f)
        self.data = list(data.values())

    def create_vocab(self):
        assert self.data is not None
        ctr = Counter()
        for d in self.data:
            ctr.update(d)
        vocab = list(ctr.keys())
        self.vocab = self.special_tokens + vocab
        self.save_vocab()

    def load_vocab(self):
        with open(self.vocab_path, 'r') as f:
            vocab = []
            for line in f:
                try:
                    vocab.append(int(line.strip()))
                except:
                    vocab.append(line.strip())
        self.vocab = vocab

    def save_vocab(self):
        assert self.vocab is not None
        with open(self.vocab_path, 'w') as f:
            for word in self.vocab:
                f.write(str(word) + '\n')


In [10]:
data_path = './data/train.pkl'  # vocab 파일을 제작할 파일 (train set)
vocab_path = './data/vocab.txt'  # 만들어진 vocab 파일의 위치  

# Create vocab.txt
vocab_module = Vocab(vocab_path)
vocab_module.load_data(data_path)
vocab_module.create_vocab()
log_vocab = vocab_module.vocab

In [11]:
log_vocab

['<PAD>',
 '<BOS>',
 '<EOS>',
 '<UNK>',
 0,
 3,
 17,
 18,
 6,
 11,
 7,
 14,
 16,
 2,
 8,
 4,
 9,
 15,
 5,
 10,
 13,
 1,
 12]

### 1-3) 데이터 전처리 3 : Pytorch Custom Datasets

파이토치는 데이터를 모델로 넣어주기 위해서는 torch.utils.data의 [Dataset 모듈](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset)을 이용합니다.  
Dataset 클래스를 상속받은 뒤 해당 클래스 내부에 모델에 넘어갈 input의 형태로 전처리를 수행합니다. 그 후 데이터를 한 개씩 반환합니다.    
Dataset 클래스를 상속받을 때 꼭 지켜야 할 점은 해당 모듈에 아래 두 가지 메소드를 오버라이딩하는 것입니다.   

```python
__len__()        # 한 개씩 반환할 데이터들의 총 개수를 반환합니다.

__getitem__()    # 매 iteration마다 한 개씩 반환할 데이터를 설정합니다.
```   

이렇게 만들어진 dataset 모듈은 torch.utils.data의 [DataLoader 모듈](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)의 인자로 이용됩니다.  

위의 Dataset의 역할이 데이터 전처리를 통해 데이터가 모델에 들어갈 수 있는 형태로 변환하고 한 개씩 내보내는 역할을 수행했다면  
DataLoader 모듈은 한 개씩 나온 데이터들을 미니배치 형태로 묶어 모델에 넘겨주는 역할을 합니다.  
이 과정에서 DataLoader 모듈은 데이터를 shuffle하거나 여러 개의 GPU를 이용할 경우 분산처리를 돕는 등 효율적인 배치 학습을 돕는 역할을 수행합니다.   

또한 DataLoader 모듈은 [collate_fn](https://pytorch.org/docs/stable/data.html#working-with-collate-fn) 인자를 통해 사용자가 직접 만든 배치 함수를 인자로 받을 수 있습니다. 즉, 사용자가 모델에 들어갈 배치의 형태를 설정할 수 있습니다.  
특히 RNN 계열의 모델을 사용한다면 일반적으로 input 데이터의 길이들이 모두 다르기 때문에 해당 기능을 통해 padding을 구현해 배치 내의 데이터들의 사이즈를 동일하게 설정해야 합니다.  

예를 들어 아래처럼 배치 안에 다섯 개의 데이터가 있는 경우 이를 동일한 길이로 맞춰주기 위해 가장 긴 문장에 맞게 길이를 맞춰줘야 합니다.   

![RNNpadding](./img/3_rnnpadding.JPG)  

자세한 예시는 코드를 통해 확인하도록 하겠습니다.  


---
# 2. RNN Log Prediction 실습  
간단한 모델부터 시작해 보겠습니다.  
본 모델은 로그 시퀀스를 input으로 받은 후 시퀀스 내부에서 윈도우 사이즈만큼을 받아 다음에 등장할 로그 키에 대한 예측을 수행합니다.  
아래는 윈도우 사이즈가 4인 예시입니다.  
<img src="./img/4_rnnprediction.JPG" width="800" height="400">   


### Dataset
아래는 Dataset을 상속받아 한 개의 데이터를 반환하는 custom dataset을 만드는 코드입니다.

**추가(11/20)**  
위에서 정의한 vocab을 이용하는 경우와 이용하지 않는 경우 두 가지가 있습니다.  
1. vocab을 이용하지 않는 경우 : 실습때와 마찬가지로 log key 각각에 4를 더해 이용합니다.  
2. vocab을 이용하는 경우 : 위에서 정의한 vocab을 이용해 index를 구합니다.  

In [13]:
# vocab 이용하지 않는 경우 (실습 때 한 내용)
sample_log = list(test.items())[0][1]
eos = 3

print("원본 데이터: ", sample_log)
print("\n위에서 정의한 log_vocab을 이용하지 않고 4를 더한 후 eos token을 추가한 값 :", [int(l) + 4 for l in sample_log] + [eos])
print("실습 때 진행했던 내용이 이와 동일합니다. ")

원본 데이터:  [0, 0, 0, 3, 17, 18, 17, 18, 17, 18, 6, 6, 6, 14, 14, 14, 16, 16, 16]

위에서 정의한 log_vocab을 이용하지 않고 4를 더한 후 eos token을 추가한 값 : [4, 4, 4, 7, 21, 22, 21, 22, 21, 22, 10, 10, 10, 18, 18, 18, 20, 20, 20, 3]
실습 때 진행했던 내용이 이와 동일합니다. 


In [16]:
print("\n위에서 정의한 log_vocab을 이용한 후 eos token을 추가한 값 :", [log_vocab.index(int(l)) for l in sample_log] + [eos])
print("로그 키들이 위 log_vocab의 index에 따라 변경됩니다.")
print("이 데이터셋은 데이터가 모두 숫자로 이루어져있기 때문에 위처럼 4를 더해도 무방하지만, 텍스트와 같이 숫자로 이루어지지 않은 데이터를 다룰 때도 있으므로 이러한 방식도 기억해두시기 바랍니다. ")


위에서 정의한 log_vocab을 이용한 후 eos token을 추가한 값 : [4, 4, 4, 5, 6, 7, 6, 7, 6, 7, 8, 8, 8, 11, 11, 11, 12, 12, 12, 3]
로그 키들이 위 log_vocab의 index에 따라 변경됩니다.
이 데이터셋은 데이터가 모두 숫자로 이루어져있기 때문에 위처럼 4를 더해도 무방하지만, 텍스트와 같이 숫자로 이루어지지 않은 데이터를 다룰 때도 있으므로 이러한 방식도 기억해두시기 바랍니다. 


In [17]:
class LogPredLoader(Dataset):
    """ 원본 데이터를 받아 전처리 후 한 개씩 반환하는 모듈 """
    def __init__(self, data_path, vocab, window_size):
        self.data_path = data_path
        self.window_size = window_size + 1
        self.blk_id, self.data = self.data_load()
        # Special tokens 정의 (첫 번째 실습에는 <PAD>나 <BOS>가 쓰이지 않지만 통일성을 위해 유지합니다.)
        self.pad, self.bos, self.eos, self.unk = (vocab.index(v) for v in ['<PAD>', '<BOS>', '<EOS>', '<UNK>'])

    def __len__(self):
        # 전체 데이터셋의 길이 반환
        return len(self.data)

    def __getitem__(self, item):
        # Special token이 4개 추가되었으므로 log key에 4씩 더하기
        rnn_input = [int(l) + 4 for l in self.data[item]] + [self.eos] # eos 추가
        """
        vocab을 이용하는 경우 (위 방식과 이 방식은 embedding matrix의 순서만 다를 뿐 모두 동일합니다. )
        rnn_input = [vocab.index(int(l)) for l in self.data[item] + [self.eos]]
        """
        rnn_input, label = list(zip(*[i for i in self.window(rnn_input)]))
        return self.blk_id[item], len(rnn_input), rnn_input, label  # block_id와 로그 길이, rnn의 input, 그리고 해당 input을 통해 예측할 target

    def data_load(self):
        # 데이터를 불러오는 함수
        with open(self.data_path, 'rb') as f:
            data = pickle.load(f)
        blk, logs = list(zip(*data.items()))
        return blk, logs
    
    def window(self, sequence):
        # 시퀀스를 window size별로 나누고 다음에 등장할 로그키를 예측하는 함수(generator)
        # ex) window size = 3 인 경우,
        # 8  3  4  5  EOS  =>  ((8, 3, 4), 5) , ((3, 4, 5), EOS)
        it = iter(sequence)
        result = tuple(islice(it, self.window_size))
        if len(result) == self.window_size:
            yield result[:-1], result[-1]
        else:
            diff = self.window_size - len(result)
            yield tuple([0] * diff + list(result[:-1])), result[-1]
            
        for elem in it:
            result = result[1:] + (elem,)
            yield result[:-1], result[-1]
    
    def batch_sequence(self, batch):
        """ 
        사용자가 직접 제작하는 배치 함수. 이 함수는 DataLoader의 collate_fn의 인자로 들어가게 됩니다.
        편의상 Dataset 모듈 내에 함수를 만들었지만 꼭 이 모듈 내에 만들 필요는 없습니다.  
        주의할 점은 이처럼 배치 함수를 만든다면 꼭 torch.tensor()함수를 이용해서 데이터를 tensor 형식으로 바꿔주고 반환해야 한다는 것입니다.
        그리고 항상 차원에 주의하세요. 기본적으로 batch 내의 input data들은 길이가 모두 다릅니다. 
        만약 collate_fn을 이용하지 않는 경우 길이가 같다면 상관 없지만 이번 실습 데이터와 같이 길이가 다른 input data들은 가장 짧은 길이의 데이터를 기준으로 잘리게 됩니다.
        따라서 본 함수를 통해 데이터 별 길이가 다른 경우 이를 맞춰주어야 합니다.
        자연어같은 경우 일반적으로 zero padding을 이용합니다.(다음 실습인 RNN AutoEncoder에서 이 방식을 이용합니다.)
        본 실습에서는 모든 데이터를 한 개의 리스트로 이어붙힙니다. 아래 예시를 통해 확인하실 수 있습니다.  
        """
        # batch 차원 : Input length x Batch size
        
        # 배치를 다시 나누기
        blk_id, lengths, rnn_input, label = [d for d in list(zip(*batch))]  # rnn_input의 차원 : Batch size x Input length
        """ 
        zip함수를 이용한 위의 방식은
        
        blk_id = [d[0] for d in batch]
        lengths = [d[1] for d in batch]
        rnn_input = [d[2] for d in batch]
        label = [d[3] for d in batch]
        
        와 동일합니다.
        """
        
        # 배치 내의 input들을 하나로 합치고 torch.tensor로 형 변환 (blk_id와 lengths는 모델에 직접적으로 들어가는 input이 아니기 때문에 변환하지 않아도 됨)
        rnn_input = torch.tensor([x for sublist in rnn_input for x in sublist])
        label = torch.tensor([x for sublist in label for x in sublist])
        
        # return하는 rnn_input의 차원 : Batch size x Input length
        
        # batch size : Batch 내의 모든 window 수 (ex. batch가 3개의 데이터로 이루어졌고, 각각의 데이터는 5, 7, 10 개의 window로 이루어졌다면 batch size : 5 + 7 + 10 = 12)
        # input length : window size
        return blk_id, lengths, rnn_input, label
    

In [18]:
""" 데이터셋의 첫 세 개의 데이터를 통한 비교 """

""" 1. 원본 데이터 """

with open(data_path, 'rb') as f:
    data = pickle.load(f)
blk, logs = list(zip(*data.items()))

eos = 2

print('first data :', [int(l) + 4 for l in logs[0]] + [eos])

print('second data :', [int(l) + 4 for l in logs[1]] + [eos])

print('third data :', [int(l) + 4 for l in logs[2]] + [eos])

first data : [4, 4, 4, 7, 21, 22, 21, 22, 10, 10, 10, 21, 22, 2]
second data : [7, 4, 4, 4, 10, 10, 10, 21, 22, 21, 22, 21, 22, 15, 11, 11, 15, 11, 11, 15, 11, 11, 18, 18, 18, 20, 20, 20, 2]
third data : [4, 4, 7, 4, 21, 22, 21, 22, 21, 22, 10, 10, 10, 2]


In [19]:
blk[:3]

('blk_-1000002529962039464',
 'blk_-100000266894974466',
 'blk_-1000007292892887521')

In [20]:
""" 2. Dataset 모듈이 데이터를 반환하는 예시 (block_id, log, length) """

dataset = LogPredLoader(data_path, log_vocab, 3)
# 첫 번째 데이터 예시 (window size = 3)
dataset.__getitem__(0)

('blk_-1000002529962039464',
 11,
 ((4, 4, 4),
  (4, 4, 7),
  (4, 7, 21),
  (7, 21, 22),
  (21, 22, 21),
  (22, 21, 22),
  (21, 22, 10),
  (22, 10, 10),
  (10, 10, 10),
  (10, 10, 21),
  (10, 21, 22)),
 (7, 21, 22, 21, 22, 10, 10, 10, 21, 22, 2))

In [21]:
# 두 번째 데이터 예시, 첫번째와 길이가 다릅니다.
dataset.__getitem__(1)

('blk_-100000266894974466',
 26,
 ((7, 4, 4),
  (4, 4, 4),
  (4, 4, 10),
  (4, 10, 10),
  (10, 10, 10),
  (10, 10, 21),
  (10, 21, 22),
  (21, 22, 21),
  (22, 21, 22),
  (21, 22, 21),
  (22, 21, 22),
  (21, 22, 15),
  (22, 15, 11),
  (15, 11, 11),
  (11, 11, 15),
  (11, 15, 11),
  (15, 11, 11),
  (11, 11, 15),
  (11, 15, 11),
  (15, 11, 11),
  (11, 11, 18),
  (11, 18, 18),
  (18, 18, 18),
  (18, 18, 20),
  (18, 20, 20),
  (20, 20, 20)),
 (4,
  10,
  10,
  10,
  21,
  22,
  21,
  22,
  21,
  22,
  15,
  11,
  11,
  15,
  11,
  11,
  15,
  11,
  11,
  18,
  18,
  18,
  20,
  20,
  20,
  2))

In [22]:
""" 3. collate_fn을 이용하지 않았을 때의 예시 """
# 기본적으로 (Input data length x Batch size) 차원 형태로 데이터를 반환합니다.
# !주의 : 배치 안에 있는 input data들의 의 길이가 모두 다를 경우 가장 짧은 데이터에 길이가 맞춰지게 됩니다.
# 아래 예시를 보면 첫 세 개의 데이터 중 가장 긴 두 번째 데이터는 길이가 잘린 것을 확인할 수 있습니다.
dl_no_collate = DataLoader(dataset, batch_size=2)
blk, lengths, input_, label = next(iter(dl_no_collate))
input_

[[tensor([4, 7]), tensor([4, 4]), tensor([4, 4])],
 [tensor([4, 4]), tensor([4, 4]), tensor([7, 4])],
 [tensor([4, 4]), tensor([7, 4]), tensor([21, 10])],
 [tensor([7, 4]), tensor([21, 10]), tensor([22, 10])],
 [tensor([21, 10]), tensor([22, 10]), tensor([21, 10])],
 [tensor([22, 10]), tensor([21, 10]), tensor([22, 21])],
 [tensor([21, 10]), tensor([22, 21]), tensor([10, 22])],
 [tensor([22, 21]), tensor([10, 22]), tensor([10, 21])],
 [tensor([10, 22]), tensor([10, 21]), tensor([10, 22])],
 [tensor([10, 21]), tensor([10, 22]), tensor([21, 21])],
 [tensor([10, 22]), tensor([21, 21]), tensor([22, 22])]]

In [23]:
""" 4. collate_fn의 인자로 제작한 배치 함수를 이용하는 예시 """
# 잘리는 데이터 없이 모두 이용되는 것 확인

dl_collate = DataLoader(dataset, 2, collate_fn=dataset.batch_sequence)
next(iter(dl_collate))
blk, lengths, input_, label = next(iter(dl_collate))
input_

tensor([[ 4,  4,  4],
        [ 4,  4,  7],
        [ 4,  7, 21],
        [ 7, 21, 22],
        [21, 22, 21],
        [22, 21, 22],
        [21, 22, 10],
        [22, 10, 10],
        [10, 10, 10],
        [10, 10, 21],
        [10, 21, 22],
        [ 7,  4,  4],
        [ 4,  4,  4],
        [ 4,  4, 10],
        [ 4, 10, 10],
        [10, 10, 10],
        [10, 10, 21],
        [10, 21, 22],
        [21, 22, 21],
        [22, 21, 22],
        [21, 22, 21],
        [22, 21, 22],
        [21, 22, 15],
        [22, 15, 11],
        [15, 11, 11],
        [11, 11, 15],
        [11, 15, 11],
        [15, 11, 11],
        [11, 11, 15],
        [11, 15, 11],
        [15, 11, 11],
        [11, 11, 18],
        [11, 18, 18],
        [18, 18, 18],
        [18, 18, 20],
        [18, 20, 20],
        [20, 20, 20]])

### Training
<left><img src="./img/5_rnnmodel.JPG" width="1000" height="600"></left>  
  
아래 코드는 위 그림처럼 데이터셋을 받아 예측을 수행하는 GRU 모델입니다. 모델은 다음과 같은 세 가지 단계로 이루어져 있습니다.    
1. 데이터를 받아 embedding을 가져오는 [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding)   
2. window size만큼의 embedding을 받아 다음에 등장할 로그를 예측하는 [gru](https://pytorch.org/docs/stable/nn.html#gru)    
3. gru의 output을 총 로그 키 수(vocab size) 차원으로 변환하는 [Linear layer](https://pytorch.org/docs/stable/nn.html#linear)   


In [24]:
class GruPredictor(nn.Module):
    def __init__(self, vocab_size, embedding_hidden_dim, gru_hidden_dim, num_hidden_layer=1):
        super(GruPredictor, self).__init__()
        
        # input sequence를 받아 저(고)차원의 vector로 변환하는 layer (lookup table)
        self.embedding = nn.Embedding(num_embeddings=vocab_size,           # embedding matrix에 들어갈 단어 차원
                                      embedding_dim=embedding_hidden_dim)  # embedding의 차원
        
        # GRU
        self.gru = nn.GRU(input_size=embedding_hidden_dim,  # GRU input 차원
                          hidden_size=gru_hidden_dim,       # GRU output 차원
                          num_layers=num_hidden_layer,      # GRU layer 수 
                          batch_first=True)                 # 기본적으로 GRU의 input은 (input length x batch size)입니다.
                                                            # batch_first=True를 설정할 시 (batch size x input length) 차원의 input을 받습니다.
        
        # GRU의 결과물을 받아 vocab size만큼 축소하는 fully connected layer (matrix)
        self.fc = nn.Linear(in_features=gru_hidden_dim,  # input hidden (or just row of matrix)
                           out_features=vocab_size)     # output hidden (or just column of matrix)
    
    def forward(self, x):
        """ 
        Dimension 표현 정리
        
        B : batch size
        L : Input length (sequence length)
        h : embedding size   / 128
        H : GRU hidden dim   / 256
        V : vocab size   / 23
        
        """
        # dimension of x : (B x L)
        
        # (B x L) ==embedding==> (B x L x h)
        x = self.embedding(x) 
        
        # (B x L x 128) ==GRU==> (B x L x 256), (1 x B x 256)
        out, hidden = self.gru(x)  
        
        # (B x L x H) ==transpose==> (L x B x 256) ==last timestep==> (B x 256)
        out = out.transpose(1, 0)[-1]
        
        # (B x 256) ==fc layer==> (B x 23)
        out = self.fc(out)  
        
        # dimension of out : (B x V)
        return out       

In [25]:
""" 모델이 작동하는지 확인 """
vocab_size = len(log_vocab)
padding_idx = log_vocab.index(0)
embedding_hidden_dim = 128
gru_hidden_dim = 256

# 모델 테스트
model = GruPredictor(vocab_size, embedding_hidden_dim, gru_hidden_dim)
print(model)

GruPredictor(
  (embedding): Embedding(23, 128)
  (gru): GRU(128, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=23, bias=True)
)


In [26]:

# 임시 데이터
dataloader = DataLoader(dataset, batch_size=3, collate_fn=dataset.batch_sequence)
blk, lengths, input_, label = next(iter(dataloader))

print("\ninput data dimension : ", input_.shape)


input data dimension :  torch.Size([48, 3])


In [27]:
input_

tensor([[ 4,  4,  4],
        [ 4,  4,  7],
        [ 4,  7, 21],
        [ 7, 21, 22],
        [21, 22, 21],
        [22, 21, 22],
        [21, 22, 10],
        [22, 10, 10],
        [10, 10, 10],
        [10, 10, 21],
        [10, 21, 22],
        [ 7,  4,  4],
        [ 4,  4,  4],
        [ 4,  4, 10],
        [ 4, 10, 10],
        [10, 10, 10],
        [10, 10, 21],
        [10, 21, 22],
        [21, 22, 21],
        [22, 21, 22],
        [21, 22, 21],
        [22, 21, 22],
        [21, 22, 15],
        [22, 15, 11],
        [15, 11, 11],
        [11, 11, 15],
        [11, 15, 11],
        [15, 11, 11],
        [11, 11, 15],
        [11, 15, 11],
        [15, 11, 11],
        [11, 11, 18],
        [11, 18, 18],
        [18, 18, 18],
        [18, 18, 20],
        [18, 20, 20],
        [20, 20, 20],
        [ 4,  4,  7],
        [ 4,  7,  4],
        [ 7,  4, 21],
        [ 4, 21, 22],
        [21, 22, 21],
        [22, 21, 22],
        [21, 22, 21],
        [22, 21, 22],
        [2

In [29]:
# 모델에 임시 데이터를 넣었을 때 output 확인
out = model(input_)
print("model output dimension : ", out.shape)

model output dimension :  torch.Size([48, 23])


In [30]:
out

tensor([[ 0.0173, -0.0892, -0.0248,  ..., -0.1307,  0.1593, -0.0635],
        [ 0.0545,  0.0436,  0.0320,  ..., -0.0930, -0.1354, -0.0510],
        [ 0.1085,  0.0894,  0.0870,  ..., -0.0378,  0.0553,  0.0605],
        ...,
        [-0.0744,  0.0546,  0.1387,  ...,  0.1084, -0.0947,  0.0139],
        [-0.1052,  0.0636,  0.1310,  ...,  0.1770, -0.2454,  0.0804],
        [-0.0973,  0.0384,  0.1235,  ...,  0.1827, -0.2894,  0.1390]],
       grad_fn=<AddmmBackward>)

In [21]:
""" 실제 학습 예시 """
train_data_path = './data/train.pkl'  # train_data set
val_data_path = './data/test.pkl' # validation data set
vocab_path = './data/vocab.txt'  # 만들어진 vocab 파일의 위치  

epochs = 10
evaluation_step = 1
window_size = 8
batch_size = 512
num_hidden_layer = 2
embedding_hidden_dim = 32
gru_hidden_dim = 64

# vocab 로드
vocab = Vocab(vocab_path).vocab
vocab_size = len(vocab)

# 데이터셋 로드
train_dataset = LogPredLoader(train_data_path, vocab, window_size=window_size)
train_dataloader = DataLoader(train_dataset, 
                              batch_size=batch_size, 
                              shuffle=True,    # 데이터셋의 순서를 shuffle
                              collate_fn=dataset.batch_sequence,
                              pin_memory=True,
                              drop_last=True)

val_dataset = LogPredLoader(val_data_path, vocab, window_size=window_size)
val_dataloader = DataLoader(val_dataset,
                            batch_size=batch_size, 
                            collate_fn=dataset.batch_sequence,
                            pin_memory=True)
print("dataset loaded")

dataset loaded


In [22]:
# 모델 로드
model = GruPredictor(vocab_size, embedding_hidden_dim, gru_hidden_dim, num_hidden_layer)

# optimizer 설정
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.005)

# learning rate scheduler 설정
total_step = len(train_dataloader) * epochs
scheduler = CosineAnnealingLR(optimizer, T_max=total_step)
# warmup scheduler (https://github.com/ildoonet/pytorch-gradual-warmup-lr)
warmup_scheduler = GradualWarmupScheduler(optimizer,
                                          multiplier=10,                  
                                          total_epoch=total_step * 0.01,
                                          after_scheduler=scheduler)

# loss 함수 설정
loss_fn = nn.CrossEntropyLoss()

# gpu 이용
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)  # 모델이 gpu 연산을 이용하도록 설정               

GruPredictor(
  (embedding): Embedding(23, 32)
  (gru): GRU(32, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=23, bias=True)
)

In [55]:
print("training starts!")
    
best_val_loss = 1e+10

for epoch in range(epochs):
    
    train_loss = 0
    train_acc = 0

    val_loss = 0
    val_acc = 0
    
    for step, batch in enumerate(train_dataloader):
        # 모델을 train모드로 설정 (dropout등의 함수로 인해 반드시 train과 eval을 잘 구분해야 합니다.)
        model.train()
        
        # 모델이 gpu 연산을 이용하므로 input 데이터가 gpu 메모리에 올라가도록 설정
        rnn_input, label = map(lambda x: x.to(device), batch[2:])  
        
        # output 계산
        output = model(rnn_input)
        
        # loss 계산
        loss = loss_fn(output, label)  # output : (B x V)
                                       # label : (B)
        # gradient 계산
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)  # gradient explosion을 방지하기 위한 gradient clipping
        
        # optimizer 계산
        optimizer.step()
        warmup_scheduler.step() # learning rate update
        
        # gradient 초기화
        model.zero_grad()
        
        # 모델의 예측값
        predict = output.max(dim=1)[1]
        
        # 예측 정확도 계산
        batch_acc = (predict == label).float().sum() / len(predict)
        
        train_loss += loss.item()
        train_acc += batch_acc.item()
    
    # validation

    for val_step, val_batch in enumerate(val_dataloader):
        # 모델을 eval로 설정
        model.eval()

        rnn_input, label = map(lambda x: x.to(device), val_batch[2:])  

        # gradient가 흐르지 않도록 설정 : 본 설정을 통해 불필요한 계산을 생략하게 됩니다.
        with torch.no_grad():

            # output 계산
            output = model(rnn_input)

            loss = loss_fn(output, label)

            predict = output.max(dim=1)[1]

            batch_acc = (predict == label).float().sum() / len(predict)

            val_loss += loss.item()
            val_acc += batch_acc.item()
    
    total_val_loss = val_loss / (val_step + 1)
        
    print("epoch : {}  /  {}  |  train_loss : {:.4f}, train_acc : {:.4f}, val_loss : {:.4f}, val_acc : {:.4f}".format(epoch, epochs, 
                                                                                                                      train_loss / (step + 1), 
                                                                                                                      train_acc / (step + 1), 
                                                                                                                      total_val_loss, 
                                                                                                                      val_acc / (val_step + 1))
    if total_val_loss < best_val_loss:
        torch.save(model.state_dict(), 'bestmodel.bin')
        best_val_loss = total_val_loss
        print('best model saved')


training starts!
epoch : 0  /  10  |  train_loss : 0.3042, train_acc : 0.8907, val_loss : 0.4949, val_acc : 0.8715
best model saved
epoch : 1  /  10  |  train_loss : 0.2546, train_acc : 0.9048, val_loss : 0.4569, val_acc : 0.8785
best model saved
epoch : 2  /  10  |  train_loss : 0.2499, train_acc : 0.9059, val_loss : 0.4909, val_acc : 0.8756
epoch : 3  /  10  |  train_loss : 0.2532, train_acc : 0.9052, val_loss : 0.4535, val_acc : 0.8797
best model saved
epoch : 4  /  10  |  train_loss : 0.2476, train_acc : 0.9066, val_loss : 0.4615, val_acc : 0.8815
epoch : 5  /  10  |  train_loss : 0.2468, train_acc : 0.9069, val_loss : 0.4518, val_acc : 0.8787
best model saved
epoch : 6  /  10  |  train_loss : 0.2414, train_acc : 0.9082, val_loss : 0.4495, val_acc : 0.8818
best model saved
epoch : 7  /  10  |  train_loss : 0.2391, train_acc : 0.909, val_loss : 0.4447, val_acc : 0.8829
best model saved
epoch : 8  /  10  |  train_loss : 0.237, train_acc : 0.9096, val_loss : 0.4408, val_acc : 0.8843
b

### Inference
test data를 통해 inference를 수행합니다. 학습과 검증에 이용한 train과 validation data가 정상적인 로그로만 이루어진 것과 달리 test data는 비정상 데이터가 섞여 있습니다.  
아래 그림 우측 하단에서 anomaly score를 산출하는 파트입니다.  
학습된 gru를 통해 테스트 데이터에 대해 anomaly score를 산출하고 결과를 확인해보겠습니다.  
<left><img src="./img/2_rnnlm.JPG" width="1200" height="600"></left>  

In [23]:
# inference
test_data_path = './data/test.pkl'

test_dataset = LogPredLoader(test_data_path, vocab, window_size=window_size)
test_dataloader = DataLoader(test_dataset, 
                             batch_size=batch_size, 
                             collate_fn=dataset.batch_sequence,
                             pin_memory=True)

# train할 때와 동일한 파라미터로 모델을 로드합니다.
infer_model = GruPredictor(vocab_size, embedding_hidden_dim, gru_hidden_dim, num_hidden_layer).to(device)

# bestmodel parameter 로드
best_params = torch.load('./bestmodel.bin')
model.load_state_dict(best_params)

<All keys matched successfully>

In [24]:
# 결과물을 저장할 list
outputs = []

for step, batch in enumerate(test_dataloader):
    # 모델을 eval모드로 설정 
    model.eval()
    
    # inference에 이용할 block_id와 lengths 로드
    block_id, lengths = batch[:2]
    rnn_input, label = map(lambda x: x.to(device), batch[2:])  

    # output 계산
    output = model(rnn_input)
    
    # 모델의 예측값 계산
    predict = output.max(dim=1)[1]
    
    # predict와 target을 길이 별로 자르기
    pred_it = iter(predict.tolist())
    pred = [list(islice(pred_it, l)) for l in lengths]
    
    true_it = iter(label.tolist())
    target = [list(islice(true_it, l)) for l in lengths]
    
    # accuracy 계산
    result = [(np.array(p) == np.array(t)).sum() / len(p) for p, t in zip(pred, target)]
    outputs += [(b, l) for b, l in zip(block_id, result)]

In [25]:
outputs[:10]

[('blk_-1000195927844309648', 1.0),
 ('blk_-1000245396392748444', 0.9230769230769231),
 ('blk_-1000285592763698141', 0.75),
 ('blk_-1000297946873432694', 1.0),
 ('blk_-1000321454365365927', 0.9230769230769231),
 ('blk_-1000495798604346871', 0.8947368421052632),
 ('blk_-1000723577943457888', 0.0),
 ('blk_-1000775598195209979', 0.6521739130434783),
 ('blk_-1000804803887048752', 0.9166666666666666),
 ('blk_-1000973735807259699', 0.8)]

In [26]:
anomaly_label = pd.read_csv('./data/anomaly_label.csv')
anomaly_label.head()

,BlockId,Label
0,blk_-1608999687919862906,Normal
1,blk_7503483334202473044,Normal
2,blk_-3544583377289625738,Anomaly
3,blk_-9073992586687739851,Normal
4,blk_7854771516489510256,Normal


In [33]:
# block id와 label을 매칭하기 위해 dictionary 형태로 변환
blk2label = anomaly_label.set_index('BlockId').to_dict()['Label']

In [34]:
# 점수가 높은 block_id와 낮은 block_id 한 개씩 확인해보기
print(blk2label['blk_-1000195927844309648'])
print(blk2label['blk_-1000723577943457888'])

# 이상치 판단은 threshold를 설정해 threshold보다 높으면 정상, 낮으면 비정상으로 할 수 있습니다. threshold는 사용자가 지정합니다.
# threshold에 무관하게 모델의 전체적인 이상치 탐지 성능을 평가하려면 AUROC를 통해 측정할 수 있습니다. 이 부분은 생략하도록 하겠습니다.

Normal
Anomaly


---
# 2. RNN AutoEncoder 실습  
다음은 로그 시퀀스를 input으로 받은 후 동일한 로그 시퀀스를 target으로 예측하는 RNN AutoEncoder입니다.  
<img src="./img/1_rnnae.JPG" width="1000" height="600">   
모델의 input은 아래 세 개 입니다.  

```python
encoder_input  # encoder의 GRU에 들어가는 log sequence (맞춰야 할 target으로도 이용됩니다.)  

decoder_input  # decoder의 GRU에 들어가는 log sequence
               # decoder는 decoder input과 encoder의 hidden state를 통해 원본 문장을 예측합니다.
    
input_mask     # padding이 된 곳을 구분하기 위한 mask
               # 이를 통해 attention이 padding에서 계산되는 것을 방지합니다.
```  

encdoer_input은 마지막에 시퀀스의 끝을 알리는 EOS토큰이 들어가며 decoder_input 시작할 때 시퀀스의 시작을 알리는 BOS토큰이 들어갑니다.  

Dataset 모듈에서는 encoder와 decoder input을 정의해줍니다.  
그리고 DataLoader 모듈의 collate_fn에 이용할 custom batch 함수에서 패딩과 input_mask를 만듭니다.  

In [31]:
class LogLoader(Dataset):
    def __init__(self, data_path, vocab):
        self.data_path = data_path
        self.blk_id, self.data, self.lengths = self.data_load()
        # Special tokens 정의
        self.pad, self.bos, self.eos, self.unk = (vocab.index(v) for v in ['<PAD>', '<BOS>', '<EOS>', '<UNK>'])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        encoder_input, decoder_input = self.preprocess(self.data[item])
        return self.blk_id[item], encoder_input, decoder_input, self.lengths[item]

    def data_load(self):
        with open(self.data_path, 'rb') as f:
            data = pickle.load(f)
        blk, logs = list(zip(*data.items()))
        lengths = [len(d) + 1 for d in logs]
        return blk, logs, lengths

    def preprocess(self, log):
        # Special token 추가를 위해 input 조정
        log = [int(l) + 4 for l in log]
        # eos, bos 추가
        encoder_input, decoder_input = log + [self.eos], [self.bos] + log
        return encoder_input, decoder_input

    def padding(self, log, max_len):
        return [l + [self.pad] * (max_len - len(l)) for l in log]  # log의 길이보다 길고, max_len보다 짧은 부분을 0으로 패딩

    def batch_sequence(self, batch):
        blk_id, encoder_input, decoder_input, lengths = [d for d in list(zip(*batch))]
        assert len(encoder_input) == len(decoder_input) and len(encoder_input) == len(lengths) # 조건에 맞지 않을 경우 경고를 반환하는 함수. 편의를 위해 이용

        max_seq_length = max(lengths) # 배치 안에서 가장 긴 문장을 확인

        # 남은 부분을 패딩 처리하고 torch.tensor를 이용해 형 변환
        encoder_input, decoder_input = torch.tensor([self.padding(log, max_seq_length)
                                                     for log in (encoder_input, decoder_input)])
        # attention을 위해 mask를 제작합니다. 원본 데이터와 길이는 같지만 원본 데이터에서 padding인 부분은 0, 아닌 부분은 1로 구성되어 있습니다.  
        input_mask = torch.ones_like(encoder_input).masked_fill(encoder_input == self.pad, 0)
        assert len(decoder_input[0]) == max_seq_length

        return blk_id, input_mask, encoder_input, decoder_input

In [32]:
""" 1. Dataset 모듈이 데이터를 반환하는 예시 (block_id, log, length) """

dataset = LogLoader(data_path, log_vocab)
# 첫 번째부터 세 번째까지의 데이터 확인 
[dataset.__getitem__(i)[1] for i in range(3)]

[[4, 4, 4, 7, 21, 22, 21, 22, 10, 10, 10, 21, 22, 2],
 [7,
  4,
  4,
  4,
  10,
  10,
  10,
  21,
  22,
  21,
  22,
  21,
  22,
  15,
  11,
  11,
  15,
  11,
  11,
  15,
  11,
  11,
  18,
  18,
  18,
  20,
  20,
  20,
  2],
 [4, 4, 7, 4, 21, 22, 21, 22, 21, 22, 10, 10, 10, 2]]

In [33]:
""" 2. collate_fn의 인자로 제작한 배치 함수를 이용하는 예시 (padding 적용)"""

dl_collate = DataLoader(dataset, batch_size=3, collate_fn=dataset.batch_sequence)
next(iter(dl_collate))
blk_id, input_mask, encoder_input, decoder_input = next(iter(dl_collate))
encoder_input

tensor([[ 4,  4,  4,  7, 21, 22, 21, 22, 10, 10, 10, 21, 22,  2,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 7,  4,  4,  4, 10, 10, 10, 21, 22, 21, 22, 21, 22, 15, 11, 11, 15, 11,
         11, 15, 11, 11, 18, 18, 18, 20, 20, 20,  2],
        [ 4,  4,  7,  4, 21, 22, 21, 22, 21, 22, 10, 10, 10,  2,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [42]:
input_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]])

### Model
  
아래 코드는 위 그림처럼 데이터셋을 받아 예측을 수행하는 seq2seq autoencoder 모델입니다.  
앞서 수행했던 RNN Prediction model보다 더 복잡하므로 대략적인 설명을 드리겠습니다.  
모델은 다음과 같은 세 가지 단계로 이루어져 있습니다.  
1. 데이터를 받아 저차원으로 압축을 시키는 **Encoder**  
2. Encoder의 압축된 Hidden state와 원본 데이터를 받아 원본 데이터를 복원하는 **Decoder**  
3. Decoder에서 원본 데이터를 복원할 때 이용하는 **Attention Layer**  
4. 위 모듈들을 합친 **AutoEncoder**

**1. Encoder**   

Encoder는 bi-directional [gru](https://pytorch.org/docs/stable/nn.html#gru) 와 fc layer로 로 구성되어 있습니다.  
앞서 RNN prediction과 차이점은 gru layer에서 반대 방향의 GRU가 하나 더 존재한다는 것입니다.    
이는 gru의 인자 중 bidirectional을 True로 설정함으로써 쉽게 구현할 수 있습니다.  
앞서 모델과 다르게 gru에 dropout조건도 넣었습니다. gru의 dropout은 gru layer가 2개 이상일 때만 이용할 수 있습니다.  
gru는 timestep별 output과 마지막 timestep의 hidden state를 도출합니다. bi-directional이므로 두 개 모두 차원이 두 배가 됩니다.  
본 모델에서는 이를 decoder의 gru 차원과 맞추기 위해 해당 차원을 fc layer를 통해 반으로 축소합니다. 그러나 취향에 맞게 축소하지 않고 decoder gru의 차원을 2배로 늘려도 무방합니다.    

*참고*   
- 파이토치의 squeeze 함수는 1인 차원을 없애줍니다. 예를 들어 (4, 2, 1, 3)이라는 차원을 가진 텐서 A가 있을 때 A.squeeze(2)를 하면 3번째에 있는 1차원이 사라져 (4, 2, 3)이 됩니다. 차원이 1인 곳에만 적용 가능합니다.  
- unsqueeze 함수는 반대로 1인 차원을 생성합니다. A의 차원이 (4, 2, 3)일 때 A.unsqueeze(0)이면 0번째에 1차원을 끼워 넣습니다. 따라서 차원은 (1, 4, 2, 3)이 됩니다.  

- 파이토치의 transpose 함수는 차원의 위치를 변경합니다.  
(4, 5, 2)의 차원을 가진 tensor A에 A.transpose(2, 1)을 수행할 경우 2번째 차원과 1번째 차원의 위치가 바뀌어 (4, 2, 5)의 차원을 갖는 텐서를 반환합니다.  


In [35]:
# squeeze, transpose 예시
a = torch.tensor([1,2,3,4])

In [36]:
a.shape

torch.Size([4])

In [38]:
b = a.unsqueeze(1)
b

tensor([[1],
        [2],
        [3],
        [4]])

In [39]:
b.shape

torch.Size([4, 1])

In [40]:
c = b.transpose(1, 0)
c

tensor([[1, 2, 3, 4]])

In [41]:
c.shape

torch.Size([1, 4])

In [42]:
d = c.squeeze(0)
d

tensor([1, 2, 3, 4])

In [43]:
d.shape

torch.Size([4])

**2. Decoder**   

Decoder는 (uni directional)gru와 attention layer, 이 둘을 합치는 concat layer, 그리고 vocab size만큼 차원을 축소시키기 위한 fc layer로 구성되어 있습니다. 
Decoder gru의 특이점은 한 timestep에 대해서만 계산을 한다는 것입니다. 추후 AutoEncoder 모듈 부분을 보시면 알겠지만 decoder에서는 input의 timestep만큼 반복문을 이용합니다.  

gru는 첫 timestep에서는 encoder의 마지막 hidden state를 받습니다.  
그 다음 timestep부터는 자기 자신의 hidden state를 재귀적으로 이용합니다.  

hidden state를 받으면 decoder input과 함께 해당 timestep의 output을 반환합니다.  
이 output은 input의 결과물과 함께 attention layer에 들어가 아래와 같이 attention weight(a_t)를 반환합니다.  
<img src="./img/7_attention2.JPG" width="600" height="300">  

attention weight을 이용해 encoder output의 weighted average를 구할 수 있습니다. 이 weighted average가 context vector가 됩니다.  
<img src="./img/6_attention.JPG" width="600" height="300">   
그 후 context vector와 decoder output을 concat한 후 비선형 함수와 fully connected layer를 거쳐 최종 output을 도출합니다.  

*참고*   
- 파이토치의 bmm은 batch matrix multiplication입니다. batch 단위의 dot product를 한다고 생각하시면 됩니다.   

예시)   
A와 B의 차원이 다음과 같습니다.  
**A : (4 x 5) , B : (5 x 10)**   
- 이 때 dot product (A, B) 를 수행하면 차원은 (4 x 10)이 됩니다.  

만약 A와 B가 128개의 batch로 이루어져 있으며 가장 앞에 batch size에 대한 차원이 있다고 가정하면 차원은 다음과 같습니다.  
**A : (128 x 4 x 5), B : (128 x 5 x 10)**  
- 이 때 bmm (A, B) 를 수행하면 차원은 (128 x 4 x 10)이 됩니다.  


In [ ]:
class GruDecoder(nn.Module):
    """ GRU Decoder for AutoEncoder """

    def __init__(self, embedding_hidden_dim, gru_hidden_dim, attention_method, vocab_size):
        super(GruDecoder, self).__init__()
        self.gru_layer = nn.GRU(input_size=embedding_hidden_dim,
                                hidden_size=gru_hidden_dim)
        self.attention_layer = Attention(method=attention_method,
                                         hidden_size=gru_hidden_dim * 2)
        self.concat_layer = nn.Linear(in_features=gru_hidden_dim * 2,
                                      out_features=gru_hidden_dim)
        self.fc_layer = nn.Linear(in_features=gru_hidden_dim,
                                  out_features=vocab_size)

    def forward(self,
                decoder_embedded,  # (1 x B x h)
                encoder_outputs,  # (B x L x H)
                last_hidden,  # (1 x B x H)
                encoder_mask):  # (B x L)

        decoder_output, last_hidden = self.gru_layer(decoder_embedded, last_hidden)  # (1 x B x H), (1 x B x H)

        # Calculate attention weights
        attention_weight = self.attention_layer(decoder_output, encoder_outputs, encoder_mask)  # (B x 1 x L)
        context = attention_weight.bmm(encoder_outputs)  # (B x 1 x L) * (B x L x H) = (B x 1 x H)

        # Concatenate attention output and decoder output
        concat_input = torch.cat((decoder_output.squeeze(0), context.squeeze(1)), 1)  # (B x 2H)
        concat_output = torch.tanh(self.concat_layer(concat_input))  # (B x H)

        # Predict next word
        out = self.fc_layer(concat_output)  # (B x V)

        return out, last_hidden


**3. Attention**   
(참고: ['딥 러닝을 이용한 자연어 처리 입문'의 어텐션 파트](https://wikidocs.net/22893))

Attention layer에서는 위 링크의 가장 아래에 있는 '4. 다양한 종류의 어텐션(Attention)'에 있는 어텐션 스코어 함수 중 'general'과 'additive', 그리고 'dot' 세 가지가 구현되어 있습니다.  
![attnscore](./img/9_attention4.JPG)
- 세 방법은 위 링크 식에 맞게 배치 단위의 매트릭스 연산으로 구현했습니다.  

각각의 방법에 맞게 어텐션 스코어를 구하면 softmax 함수를 통해 어텐션 weight가 됩니다. 이를 통해 encoder input의 weighted average를 구하게 됩니다.  
여기서 특이점은 attention weight를 구하기 전 앞서 정의했던 attention mask를 통해 padding이 적용된 부분을 음의 무한대로 보내는 것입니다.(아래에선 -1e10으로 설정)  
음의 무한대가 된 부분은 softmax 함수에 들어가면 0을 반환합니다. 즉 weight가 0이 되고 추후 weighted average를 구할 때 이 부분은 무시가 됩니다.  



In [44]:
class Attention(nn.Module):
    """Implementation of various attention score function"""

    def __init__(self, method, hidden_size):
        super(Attention, self).__init__()
        self.method = method

        if self.method == 'general':
            self.attn = nn.Linear(hidden_size, hidden_size)

        elif self.method == 'concat':  # Luong(additive) Attention
            self.attn = nn.Linear(hidden_size * 2, hidden_size)
            self.v = nn.Linear(hidden_size, 1)

        elif self.method == 'dot':
            pass

    def forward(self,
                decoder_outputs,  # (1 x B x H)
                encoder_outputs,  # (B x L x H)
                encoder_mask):  # (B x L)

        attn_energies = self.score(decoder_outputs, encoder_outputs, encoder_mask)
        attn_weight = torch.softmax(attn_energies, 2)

        return attn_weight  # (B x 1 x L)

    def score(self, decoder_outputs, encoder_outputs, encoder_mask):
        """ Attention score functions """
        decoder_outputs = decoder_outputs.transpose(0, 1)
        if self.method == 'dot':
            # (B x 1 x H) x (B x H x L) = (B x 1 x L)
            energy = torch.bmm(decoder_outputs, encoder_outputs.transpose(1, 2))
        elif self.method == 'general':
            # (B x 1 x H) x (B x H x L) = (B x 1 x L)
            energy = torch.bmm(decoder_outputs, self.attn(encoder_outputs).transpose(1, 2))
        elif self.method == 'concat':
            seq_length = encoder_outputs.shape[1]
            # (B x L x H) ; (B x L x H) => (B x L x 2H)
            concat = torch.cat((decoder_outputs.repeat(1, seq_length, 1), encoder_outputs), 2)
            # (B x L x 2H) =attn=> (B x L x H) =v=> (B x L x 1)
            energy = self.v(torch.tanh(self.attn(concat))).transpose(1, 2)  # (B x 1 x L)
        else:
            raise ValueError("Invalid attention method")

        # Mask to pad token
        energy = energy.masked_fill(encoder_mask.unsqueeze(1) == 0, -1e10)  # condition을 만족하는 부분의 값을 변경

        return energy


**4. AutoEncoder**   
앞서 정의한 encoder와 decoder를 이용해 최종적인 Autoencoder를 구축합니다.  
특이점은 해당 모듈에서 embedding을 수행하며 embedding이 수행된 벡터를 encoder와 decoder에 집어넣습니다.  
그리고 decoder는 timestep만큼 반복문을 통해 수행되게 됩니다.  
마지막에는 output과 함께 loss를 계산해서 내보냅니다.  
Loss는 첫 번째 예시에서와 같이 train 과정에서 정의해도 되지만 이처럼 모듈 내에서 정의해도 무방합니다.  

In [ ]:
class GruAutoEncoder(nn.Module):
    """
    GRU AutoEncoder

    Encoder : Bi-Directional GRU
    Decoder : GRU
    """
    def __init__(self, vocab, embedding_hidden_dim, num_hidden_layer, gru_hidden_dim, device,
                 dropout_p=0.1, attention_method='dot'):
        super(GruAutoEncoder, self).__init__()
        self.device = device
        self.vocab_size = len(vocab)
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab),
                                            embedding_dim=embedding_hidden_dim,
                                            padding_idx=vocab.index('<PAD>'))
        self.encoder = BiGruEncoder(embedding_hidden_dim, gru_hidden_dim, num_hidden_layer, dropout_p)
        self.decoder = GruDecoder(embedding_hidden_dim, gru_hidden_dim, attention_method, self.vocab_size)
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=vocab.index('<PAD>'))

    def forward(self,
                encoder_mask,  # (B x L)
                encoder_input,  # (B x L)
                decoder_input, ):  # (B x L)

        batch_size, max_len = encoder_input.shape
        encoder_embedded = self.embedding_layer(encoder_input)  # (B x L x h)
        
        encoder_outputs, hidden = self.encoder(encoder_embedded=encoder_embedded)  # (B x L x H), (1 x B x H)
        
        outputs = torch.zeros(max_len, batch_size, self.vocab_size).to(self.device)  # (L x B x V)
        decoder_embedded = self.embedding_layer(decoder_input).transpose(0, 1)  # (L x B x h)
        for t in range(max_len):
            out, hidden = self.decoder(decoder_embedded=decoder_embedded[t].unsqueeze(0),
                                       encoder_outputs=encoder_outputs,
                                       last_hidden=hidden,
                                       encoder_mask=encoder_mask, )
            outputs[t] = out

        loss = self.loss_fn(outputs.view(-1, self.vocab_size),  # (L x B x V) => (L*B x V)
                            encoder_input.transpose(1, 0).reshape(-1))  # (L x B) => (L*B)

        return outputs, loss


### Training
훈련을 수행하는 train 함수는 train.py에 구현되어 있습니다.  
본 함수의 전체적인 구조는 실습 1과 유사합니다.  
다만 주피터 노트북이 아닌 cmd 창에서 학습이 되도록 구성되었습니다.(강의자가 주로 이용하는 방식입니다.)    

train.py를 통해 학습을 수행할 수 있습니다. (linux 환경의 경우 trainer.sh의 예시와 같이 쉘 스크립트를 통해 수행할 수도 있습니다.)  
인자를 조정하고 싶을 경우 argparse를 이용합니다. **train.py --변수명 변수값** 의 형식으로 입력하면 됩니다. 변수를 입력받지 않으면 설정된 기본값으로 실행됩니다.
- ex : train.py --batch_size 512 --epochs 10 --gru_hidden_dim 256  


1. python argparse package를 통해 파라미터를 입력받습니다.  
2. tqdm 패키지를 통해 progress를 표시합니다.(tqdm을 이용하면 print가 아닌 tqdm.write를 이용해야 한다는 것을 주의하세요)  
3. [mixed precision training](https://medium.com/the-artificial-impostor/use-nvidia-apex-for-easy-mixed-precision-training-in-pytorch-46841c6eed8c)이 구현되어 있습니다. (현재 리눅스 환경에서만 작동합니다.)  
4. logging을 통해 학습 상태를 표시합니다.   
5. seed를 고정하는 함수를 구현했습니다. seed를 설정하면 성능에 대한 reproduction이 가능하고 실험을 수행할 때 유용합니다.    
6. tensorboard를 통해 학습 과정을 확인할 수 있습니다. 해당 정보는 model_saved/ 폴더에 저장됩니다.  
7. 모델 학습이 완료되면 hyper_search에 결과가 저장됩니다.  